In [2]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Doc2Vec
import pickle

In [3]:
df_NM= pd.read_csv("D:\FINAL_PM_Modi_Speech_Cleaned_string.csv")

In [8]:
df_NM.head()

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...


In [4]:
df_MS= pd.read_csv("D:\PM_MANMOHAN_Speech_Cleaned_string.csv")

In [13]:
df_MS = df_MS.drop('Location', axis=1)

In [14]:
df_MS.head()

,index,Date,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...


In [18]:
df_MS.shape

(1401, 6)

In [19]:
df_NM.shape

(1182, 6)

In [15]:
meta_df = pd.concat([df_NM, df_MS])

In [17]:
meta_df.shape


(2583, 6)

In [20]:
meta_df['Cleaned_Speech_string'] = meta_df['Cleaned_Speech_string'].astype(str)

In [22]:
corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in meta_df["Cleaned_Speech_string"]]

In [23]:
len(corpus_tokenized)

2583

In [24]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_docs = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(corpus_tokenized)]

In [25]:
len(tagged_docs)

2583

In [26]:
#TRAINING THE MODEL OVER OUR DATA
model = Doc2Vec(tagged_docs, vector_size=300, window=5, min_count=5, workers=4, epochs=20)

In [27]:
import pandas as pd

# Create an empty DataFrame with the desired column names
df_data = pd.DataFrame(columns=['index', 'vector'])

# Iterate over the corpus_tokenized list and generate a vector for each element
for i, speech in enumerate(corpus_tokenized):
    vector = model.infer_vector(speech)
    # Store the index number and vector in a dictionary
    data = {'index': i, 'vector': vector}
    # Add the dictionary as a new row in the DataFrame
    df_data.loc[len(df_data)] = data


In [28]:
for i in range(300):
    col_name = f'dim_{i+1}'
    df_data[col_name] = 0.0

# Iterate over each row in the DataFrame and assign the ith element of the vector to the corresponding column
for i, row in df_data.iterrows():
    for j, element in enumerate(row['vector']):
        col_name = f'dim_{j+1}'
        df_data.at[i, col_name] = element

C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_20000\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_data[col_name] = 0.0
C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_20000\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_data[col_name] = 0.0
C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_20000\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [29]:
df_data = df_data.drop('index', axis=1)
df_data = df_data.drop('vector', axis=1)

In [34]:
df_data.insert(0, "index", meta_df["index"])
df_data.insert(1, "Date", meta_df["Date"])

ValueError: cannot reindex on an axis with duplicate labels

In [32]:
df_data

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299,dim_300
0,-1.579309,-2.656151,0.056682,1.874645,3.436922,0.625251,-0.569035,1.916155,0.621076,0.227908,...,-0.936509,1.847846,-0.590620,0.267817,1.259360,1.834759,-1.541388,0.314627,-0.356982,1.251445
1,-0.853675,-1.213459,0.820390,0.906653,3.125370,0.020098,0.779321,1.791951,-1.426650,-2.420758,...,0.779279,0.763501,-0.085834,-0.481311,1.451109,0.788504,-0.861668,-1.310954,0.747582,2.459951
2,-1.826490,-0.875522,2.289102,-0.924871,-0.013715,-1.463894,0.261868,-0.570867,-0.776771,2.589385,...,0.946953,0.662093,2.107495,-1.520113,-2.139210,1.176933,0.830000,-1.479885,0.763641,0.549136
3,-3.005569,0.849373,-1.316612,-0.544228,0.599036,-2.564347,-1.908983,0.570318,-0.885695,0.951956,...,-0.097362,-1.657738,0.163937,0.553291,2.567361,1.029443,-0.375420,-0.108405,0.483358,0.869689
4,-0.350801,0.823885,1.551711,0.781946,0.882583,0.282651,1.436404,-0.937954,-1.826803,1.430882,...,-1.197581,0.829158,-0.511362,-1.147346,0.076391,2.683234,1.030648,-0.180534,0.293285,0.108647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,0.227182,0.809092,0.015403,0.541123,-0.317663,0.557276,-0.134409,-0.037461,-0.178637,1.755585,...,-0.282547,-0.504741,-0.116777,0.181396,0.109683,0.796428,-0.302612,-0.211387,0.781447,0.582779
2579,0.990132,1.251331,0.532488,0.581014,0.740222,-1.011377,-0.572724,0.129806,-1.009428,0.254369,...,-0.196774,-0.703163,0.660313,-1.046622,0.294051,0.233016,-1.314975,-1.541784,1.446857,-1.233088
2580,-0.353016,-0.043431,1.330329,-0.559366,1.483365,-0.133979,-0.036944,-0.064371,-0.988677,0.637097,...,0.969782,-0.469916,0.871533,0.209993,0.087369,1.475714,-1.233526,-0.302733,0.203298,0.211272
2581,1.823831,-0.731062,2.464443,0.595511,0.080946,0.228613,0.608068,-0.778040,-0.975103,-0.427016,...,0.409349,-0.679345,0.476872,-0.549376,1.096129,0.437800,-1.431932,-0.415868,-0.518859,-0.154932
